In [410]:
#import relevant module
from bs4 import BeautifulSoup, element
import requests
import pandas as pd
data = {
    "links": [],
    "name": [],
    "company": [],
    "location":[],
    "position": [],
    "salary": [],
    "level": []
}

In [398]:
def filter(soup, lst):
        if isinstance(soup, element.Tag):
            name = soup.text
            lst.append(name)
        elif not isinstance(soup, element.Tag):
             lst.append("N/A")
        else:
            lst.append("N/A")

url = "https://www.seek.com.au/plc-engineer-jobs/in-All-Australia?page=11"
test = requests.get(url)
small_soup = BeautifulSoup(test.text, "html.parser")
name = small_soup.find('li', '_1ungv2r0 _1viagsnbb _1viagsnb0 _1viagsnx')
#name = name.span
#print(name.find("a").get("href"))
print(name)


In [404]:
def extract(url):
    respone = requests.get(url)
    soup = BeautifulSoup(respone.text, "html.parser")
    cards = soup.find_all('div', '_1ungv2r0 _1viagsn4z _1viagsn4x')
    
    for card in cards:
        link = card.find('a')
        link = link.get("href")
        if link != None:
            data["links"].append("https://www.seek.com.au"+str(link))

    #recursion function to go to the next page
    next_card = soup.find('li', '_1ungv2r0 _1viagsnbb _1viagsnb0 _1viagsnx')
    while isinstance(next_card, element.Tag):
        n_link = next_card.find('a').get('href')
        return extract("https://www.seek.com.au"+str(n_link))

    #extract detail data
    for i in data["links"]:
        #extract raw html as text format
        jobs = requests.get(i)
        jobs_soup = BeautifulSoup(jobs.text, "html.parser")


        job_name = jobs_soup.find("h1", attrs={"class": "_1ungv2r0 _1viagsn4z _3h66av0 _3h66avl _1708b944 _3h66avs _3h66av21"})
        job_company = jobs_soup.find('span', attrs={"class": '_1ungv2r0 _1viagsn4z _1viagsni7 _3h66av0 _3h66av1 _3h66av21 _1708b944 _3h66ava'})
        job_details = jobs_soup.find_all("div", attrs={"class": "_1ungv2r0 _1viagsn5b _1viagsnh7 _1viagsngr _157hsn62d"})
        
        ##
        filter(job_name, data["name"])
        ##
        filter(job_company, data["company"])
        ##
        while len(job_details) < 5:
            job_details.append("N/A")
        ##
        filter(job_details[1], data["location"])
        ##
        filter(job_details[2], data["position"])
        ##
        filter(job_details[3], data["level"])
        ##
        filter(job_details[4], data["salary"])


In [411]:
url = "https://www.seek.com.au/automation-engineer-jobs/in-All-Australia"
extract(url)
df = pd.DataFrame(data)
df
df.to_csv("job_data2.csv", index = False)
